In [3]:
from pyspark.sql import SparkSession
import utils as utils
#.master("spark://spark-master:7077") \

problem_data_path = 'problematic_transactions.csv'

utils.verify_path(problem_data_path)
import os
os.getcwd()

Path exists!


'/home/jovyan/work'

In [2]:
from pyspark.sql import SparkSession

def get_spark_session(app_name="SparkApplication", master="local[*]", config_options=None):
    """
    Creates and returns a SparkSession with specified configurations.
    :param app_name: Name of the Spark application
    :param master: Spark master URL
    :param config_options: Additional configurations as a dictionary
    :return: SparkSession instance
    """
    builder = SparkSession.builder.appName(app_name).master(master)
    
    if config_options:
        for key, value in config_options.items():
            builder = builder.config(key, value)
    
    return builder.getOrCreate()

def get_jdbc_connection_properties(user, password, driver="org.postgresql.Driver"):
    """
    Returns a dictionary of JDBC connection properties.
    :param user: Database username
    :param password: Database password
    :param driver: JDBC driver class
    :return: Dictionary of connection properties
    """
    return {
        "user": user,
        "password": password,
        "driver": driver
    }

def get_psql_data(jdbc_url, table_name, user, password, jdbc_jar_path="/opt/spark/jars/postgresql-42.7.4.jar"):
    """
    Loads data from a PostgreSQL table into a Spark DataFrame.
    :param jdbc_url: JDBC URL for the PostgreSQL database
    :param table_name: Table name to load
    :param user: Database username
    :param password: Database password
    :param jdbc_jar_path: Path to the PostgreSQL JDBC driver jar
    :return: Spark DataFrame
    """
    spark = get_spark_session(
        app_name="Spark with PostgreSQL",
        config_options={"spark.jars": jdbc_jar_path}
    )
    
    connection_properties = get_jdbc_connection_properties(user, password)
    
    df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)
    return df

import utils  # Assuming this contains the verify_path function

def get_csv_data(file_path):
    """
    Loads data from a CSV file into a Spark DataFrame.
    :param file_path: Path to the CSV file
    :return: Spark DataFrame
    """
    spark = get_spark_session(app_name="CSVDataLoader")
    
    # Verify the file path exists
    utils.verify_path(file_path)
    
    # Read the CSV file into a Spark DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    return df
    
jdbc_url = "jdbc:postgresql://psql-postgres-1:5432/mydatabase"
table_name = "transactions"
user = "myuser"
password = "mysecretpassword"

psql_df = get_psql_data(jdbc_url, table_name, user, password)
print(psql_df.show())

csv_file_path = 'problematic_transactions.csv'

csv_df = get_csv_data(csv_file_path)
print(csv_df.show())


+--------------+-------+------+--------+-------------------+
|transaction_id|user_id|amount|    type|          timestamp|
+--------------+-------+------+--------+-------------------+
|    1704169733|      9|361.83|transfer|2024-01-01 20:28:53|
|    1706848193|     77| 55.28|transfer|2024-02-01 20:29:53|
|    1709353853|     69| 82.96|purchase|2024-03-01 20:30:53|
|    1712028713|     88| 36.10|transfer|2024-04-01 20:31:53|
|    1714620773|     22| 94.44|transfer|2024-05-01 20:32:53|
|    1717299233|     73|408.42|purchase|2024-06-01 20:33:53|
|    1719891293|     64|954.82|transfer|2024-07-01 20:34:53|
|    1722569753|     59|410.94|purchase|2024-08-01 20:35:53|
|    1725248213|     16|797.11|purchase|2024-09-01 20:36:53|
|    1727840273|     42|802.94|purchase|2024-10-01 20:37:53|
|    1730518733|     43| 35.93|transfer|2024-11-01 20:38:53|
|    1733114393|     69|555.90|  refund|2024-12-01 20:39:53|
|    1704170453|     55| 15.16|transfer|2024-01-01 20:40:53|
|    1706848913|     24|

In [1]:
def get_psql_data():
    jdbc_path = "/opt/spark/jars/postgresql-42.7.4.jar"
    
    # Initialize Spark session
    spark = SparkSession.builder \
        .appName("Spark with PostgreSQL") \
        .master("local[*]") \
        .config("spark.jars", jdbc_path) \
        .getOrCreate()
    
    # JDBC connection properties
    jdbc_url = "jdbc:postgresql://psql-postgres-1:5432/mydatabase"
    connection_properties = {
        "user": "myuser",
        "password": "mysecretpassword",
        "driver": "org.postgresql.Driver"
    }
    
    # Load a table into Spark
    df = spark.read.jdbc(url=jdbc_url, table="transactions", properties=connection_properties)
    return df.show()


df = get_psql_data()

In [7]:
def get_csv_data():
    from pyspark.sql import SparkSession

    # Initialize SparkSession
    spark = SparkSession.builder\
    .appName("CSVDataLoader")\
    .master("local[*]") \
    .getOrCreate()

    # Define the file path
    problem_data_path = 'problematic_transactions.csv'

    # Verify the file path exists (assuming `utils.verify_path` is a custom function)
    utils.verify_path(problem_data_path)

    # Read the CSV file into a Spark DataFrame
    df = spark.read.csv(problem_data_path, header=True, inferSchema=True)

    # Show a sample of the data (optional)
    df.show()

    # Return the Spark DataFrame
    return df
# Load CSV data using Spark
csv_df = get_csv_data()

# Perform Spark transformations
csv_df.printSchema()  # Show the schema of the DataFrame
csv_df.select("transaction_id", "amount").show()  # Select specific columns


Path exists!
+--------------+-------+--------------+--------+--------------------+
|transaction_id|user_id|        amount|    type|           timestamp|
+--------------+-------+--------------+--------+--------------------+
|    1704314682|     45|960.52 dollars|  refund|2024-01-03T12:44:42Z|
|    1706993142|     70|434.41 dollars|purchase|2024-02-03T12:45:42Z|
|    1709498802|     80|856.68 dollars|  refund|2024-03-03T12:46:42Z|
|    1712173662|     10|          NULL|transfer|2024-04-03T12:47:42Z|
|    1714765722|     88|        108.19|transfer|2024-05-03T12:44:42Z|
|    1717444182|     45|          NULL|  refund|2024-06-03T12:49:42Z|
|    1720036242|     24|        950.52|purchase|2024-07-03T12:50:42Z|
|    1720036242|     24|        950.52|purchase|2024-07-03T12:50:42Z|
|    1722714702|     51|        479.82|transfer|2024/08-03T12:51:42Z|
|    1725393162|     93|        671.08|  refund|2024-09-03T11:39:42Z|
|    1727985222|     60|505.53 dollars|purchase|2024-10-03T12:53:42Z|
|    17

+--------------+-------+------+--------+-------------------+
|transaction_id|user_id|amount|    type|          timestamp|
+--------------+-------+------+--------+-------------------+
|    1704169733|      9|361.83|transfer|2024-01-01 20:28:53|
|    1706848193|     77| 55.28|transfer|2024-02-01 20:29:53|
|    1709353853|     69| 82.96|purchase|2024-03-01 20:30:53|
|    1712028713|     88| 36.10|transfer|2024-04-01 20:31:53|
|    1714620773|     22| 94.44|transfer|2024-05-01 20:32:53|
|    1717299233|     73|408.42|purchase|2024-06-01 20:33:53|
|    1719891293|     64|954.82|transfer|2024-07-01 20:34:53|
|    1722569753|     59|410.94|purchase|2024-08-01 20:35:53|
|    1725248213|     16|797.11|purchase|2024-09-01 20:36:53|
|    1727840273|     42|802.94|purchase|2024-10-01 20:37:53|
|    1730518733|     43| 35.93|transfer|2024-11-01 20:38:53|
|    1733114393|     69|555.90|  refund|2024-12-01 20:39:53|
|    1704170453|     55| 15.16|transfer|2024-01-01 20:40:53|
|    1706848913|     24|

In [15]:
import psycopg2
conn = psycopg2.connect(
    host="psql-postgres-1",
    port=5432,
    user="myuser",
    password="mysecretpassword",
    dbname="mydatabase"
)
print("Connection successful!")

cur = conn.cursor()

cur.execute("""
    SELECT * FROM transactions;
""")

rows = cur.fetchall()

# Print each row
for i, row in enumerate(rows):
    if i == 10:
        break
    print(row)
    

# Commit the changes and close the connection
conn.commit()
conn.close()


Connection successful!
(1704169733, 9, Decimal('361.83'), 'transfer', datetime.datetime(2024, 1, 1, 20, 28, 53))
(1706848193, 77, Decimal('55.28'), 'transfer', datetime.datetime(2024, 2, 1, 20, 29, 53))
(1709353853, 69, Decimal('82.96'), 'purchase', datetime.datetime(2024, 3, 1, 20, 30, 53))
(1712028713, 88, Decimal('36.10'), 'transfer', datetime.datetime(2024, 4, 1, 20, 31, 53))
(1714620773, 22, Decimal('94.44'), 'transfer', datetime.datetime(2024, 5, 1, 20, 32, 53))
(1717299233, 73, Decimal('408.42'), 'purchase', datetime.datetime(2024, 6, 1, 20, 33, 53))
(1719891293, 64, Decimal('954.82'), 'transfer', datetime.datetime(2024, 7, 1, 20, 34, 53))
(1722569753, 59, Decimal('410.94'), 'purchase', datetime.datetime(2024, 8, 1, 20, 35, 53))
(1725248213, 16, Decimal('797.11'), 'purchase', datetime.datetime(2024, 9, 1, 20, 36, 53))
(1727840273, 42, Decimal('802.94'), 'purchase', datetime.datetime(2024, 10, 1, 20, 37, 53))


In [8]:
from pyspark.sql import SparkSession

spark.catalog.clearCache()

if 'spark' in globals():
    spark.stop()
    
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Spark Test") \
    .master("local[*]") \
    .getOrCreate()

# Create a simple DataFrame
data = [("Alice", 29), ("Bob", 35), ("Cathy", 45)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

# Perform a simple transformation: filter by age > 30
df_filtered = df.filter(df["Age"] > 30)

# Show the filtered DataFrame
df_filtered.show()

# Stop the Spark session when done
spark.stop()


+-----+---+
| Name|Age|
+-----+---+
|Alice| 29|
|  Bob| 35|
|Cathy| 45|
+-----+---+

+-----+---+
| Name|Age|
+-----+---+
|  Bob| 35|
|Cathy| 45|
+-----+---+



In [5]:
if 'spark' in globals():
    spark.stop()

NameError: name 'pyspark' is not defined

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("SimpleApp").getOrCreate()

data = [("Alice", 1), ("Bob", 2), ("Charlie", 3)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)
df.show()


+-------+---+
|   Name|Age|
+-------+---+
|  Alice|  1|
|    Bob|  2|
|Charlie|  3|
+-------+---+



In [11]:
pip install psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.7 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.
